<a href="https://colab.research.google.com/github/astepankoff/gigachat_tutorials/blob/main/Telegram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Подключение Гигачата к собственному боту в Telegram
В рамках этого туториала мы изучим два вопроса:
- Использование возможностей LangChain для управления памятью в рамках диалога;
- Взаимодействие бота и языковой модели (передача сообщения пользователя в языковую модель и отправка ответа модели в бота).
## Получение токена бота

Нам понадобится свой бот. Если вы не создавали ботов в Telegram ранее, то выполните следующие шаги:

   - Найдите в Telegram аккаунт [@BotFather](https://t.me/BotFather).
   - Напишите ему команду `/newbot`.
   - Следуйте инструкциям BotFather для выбора имени (может быть любое) и username вашего бота (это уникальный идентификатор и часть url бота, который должен содержать "bot" в своем названии).
   - После создания бота BotFather предоставит вам токен для доступа к Telegram Bot API.

## Устанавливаем требуемые библиотеки
Существует большое количество библиотек для работы с ботами Telegram. Мы в этом туториале будем использовать [pyTelegramBotAPI](https://github.com/eternnoir/pyTelegramBotAPI). Если вам превычнее работать с другой библиотекой, используйте свой вариант.

Также нам понадобится уже знакомая нам по [прошлому туториалу](https://youtu.be/eo5LyjRobic) библиотека [GigaChain](https://github.com/ai-forever/gigachain).

In [1]:
!pip install -q pyTelegramBotAPI
!pip install -q gigachain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 13.1 MB/s eta 0:00:00


## Импортируем ключи
Нам понадобится токен от бота и ключ для авторизации в Гигачат. Я дляэтого использую встроенные в Сolab секреты.

In [2]:
from google.colab import userdata
sber = userdata.get('SBER_AUTH')
bot_token = userdata.get('BOT_TOKEN')

## Работа с памятью (контекстным окном) в LangChain
Прежде чем подключать GigaChat к боту, давайте просто через взаимодействие с языковой моделью изучим возможности LangChain в плане "памяти" (хранения сообщений и/или общего контекста диалога)

Подробно про память в LangChain можно почитать в [документации](https://python.langchain.com/docs/modules/memory/).

Языковые модели не обладают памятью. Они просто генерируют новый текст на основании текста, который был передан им в качестве промпта. Текст промпта может содержать весь диалог, начиная с системного сообщения или первого сообщения пользователя. Однако тогда каждое последующее сообщение от пользователя будет предворятся все более длинным текстом истории диалога. Это чревато большим расходом токенов и грозит в какой-то момент превысить размер контекстного окна.

Чтобы не превысить размеры окна и сэкономить на расходе токенов LangChain предлагает различные типы памяти. Перечислим основные из них:
- [`ConversationBufferMemory`](https://python.langchain.com/docs/modules/memory/types/buffer) - самый простой вариант. Позволяет хранить, записывать и передавать в переменные сообщения из диалога.
- [`ConversationBufferWindowMemory`](https://python.langchain.com/docs/modules/memory/types/buffer_window) - почти тоже самое, что и `ConversationBufferMemory`, но держит в памяти только k последних пар сообщений (под парой понимается пользовательский запрос и ответ языковой модели). Получается такое своего рода скользящее окно.
- [`ConversationEntityMemory`](https://python.langchain.com/docs/modules/memory/types/entity_summary_memory) - помимо истории сообщений создает еще хранилище отдельных сущностей (entities), которые попадаются в ходе диалога. Поучается, что помимо неструктурированного текста языковая модель работает и со структурированными извлеченными сущностями.
- [`ConversationKGMemory`](https://python.langchain.com/docs/modules/memory/types/kg) - память в виде графа знаний.
- [`ConversationSummaryMemory`](https://python.langchain.com/docs/modules/memory/types/summary) - память в виде краткого содержания разговора. Сами сообщения не хранятся, просто языковая модель регулярно пополняет память новыми деталями из свежих сообщений диалога.
- [`ConversationSummaryBufferMemory`](https://python.langchain.com/docs/modules/memory/types/summary_buffer) - хранит и некоторое количество сообщений диалога, и краткое описание всей беседы в целом.
- [`ConversationTokenBufferMemory`]() - подобно `ConversationBufferWindowMemory` хранит ограниченное количество сообщений, но здесь вместо лимита по количеству сообщений лимит по количеству токенов.
- [`VectorStoreRetrieverMemory`](https://python.langchain.com/docs/modules/memory/types/vectorstore_retriever_memory) - отправляет сообщения в векторное хранилище. Там сообщения хранятся произвольно, отпраляясь в контекст диалога только если они семантически релевантны сообщению пользователя.   

### Сравнение обычной памяти и памяти с извлеченными сущностями

Для экономии времени в этом туториале мы рассмотрим лишь `ConversationBufferMemory` и `ConversationEntityMemory`.

Для удобства работы с диалогами мы используем [`ConversationChain`](https://api.python.langchain.com/en/latest/chains/langchain.chains.conversation.base.ConversationChain.html). Это удобный инструмент, чтобы соединить память и языковую модель.

#### ConversationBufferMemory

In [6]:
!pip install -U langchain-community
!pip install gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.4/69.4 kB 2.4 MB/s eta 0:00:00


In [7]:
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

llm = GigaChat(credentials=sber, verify_ssl_certs=False)
conversation1 = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

/tmp/ipython-input-2105212129.py:9: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory()
/tmp/ipython-input-2105212129.py:6: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  conversation1 = ConversationChain(


Напишем несколько сообщений языковой модели и посмотрим, что именно хранится в памяти.

In [8]:
conversation1.predict(input='Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI:

> Finished chain.


'Привет, Сергей!\n\nРад слышать, что ты снимаешь видео про GigaChat и Python — отличная тема! Если тебе нужно немного вдохновения или помощь с идеями сценария, расскажу пару вещей, которые точно пригодятся.\n\n1. **Краткий обзор GigaChat**: Можешь рассказать, как работает нейросетевой помощник и какие основные функции у него есть (например, помощь с кодом, анализ текста, генерация контента).\n   \n2. **Практический пример программирования на Python**: Покажи простой скрипт, который решает конкретную задачу (например, работа с файлами, обработка изображений или даже генерация небольших изображений). Можно показать процесс написания кода от начала до конца.\n\n3. **Советы начинающим разработчикам**: Упомяни несколько полезных ресурсов или библиотек Python, которые помогут новичкам начать писать эффективные программы.\n\n4. **Заключение и призыв к действию**: Закончись мотивирующим роликом или предложением посмотреть другие твои видео, где рассматриваются аналогичные темы.\n\nНадеюсь, эти

In [9]:
conversation1.predict(input='Какая большая языковая модель самая лучшая?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI: Привет, Сергей!

Рад слышать, что ты снимаешь видео про GigaChat и Python — отличная тема! Если тебе нужно немного вдохновения или помощь с идеями сценария, расскажу пару вещей, которые точно пригодятся.

1. **Краткий обзор GigaChat**: Можешь рассказать, как работает нейросетевой помощник и какие основные функции у него есть (например, помощь с кодом, анализ текста, генерация контента).
   
2. **Практический пример программирования на Python**: Покажи простой скрипт, который решает конкретную задачу (например, работа с файлами, обработка изображений или даже генерация небольших изобра

'[Объяснение вопроса]\n\nВопрос касается выбора лучшей большой языковой модели (LLM), и сразу стоит отметить, что понятие «лучшая» зависит от контекста и целей конкретного пользователя. Различные LLM обладают разными сильными сторонами и особенностями, подходящими для разных сценариев использования.\n\n- Для некоторых пользователей важнее высокая точность и аккуратность в ответах, особенно когда речь идет о профессиональной деятельности, связанной с юридической консультацией, медицинской диагностикой или техническим проектированием.\n- Другим же пользователям важнее креативность и гибкость, чтобы получать интересные и нестандартные решения в области творчества, развлечений и общения.\n\nПоэтому вопрос сводится к личным предпочтениям и целям, а также возможностям конкретной модели.\n\n### Сравнение нескольких популярных моделей\n\n#### 1. GigaChat (Russian)\n- **Сильные стороны:** Очень хорошо адаптирован к русскоязычным данным, культурным особенностям и российским реалиям. Легко справл

In [10]:
conversation1.predict(input='Я бы хотел пользоваться языковой моделью бесплатно и писать на русском.')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI: Привет, Сергей!

Рад слышать, что ты снимаешь видео про GigaChat и Python — отличная тема! Если тебе нужно немного вдохновения или помощь с идеями сценария, расскажу пару вещей, которые точно пригодятся.

1. **Краткий обзор GigaChat**: Можешь рассказать, как работает нейросетевой помощник и какие основные функции у него есть (например, помощь с кодом, анализ текста, генерация контента).
   
2. **Практический пример программирования на Python**: Покажи простой скрипт, который решает конкретную задачу (например, работа с файлами, обработка изображений или даже генерация небольших изобра

'# Какие бесплатные большие языковые модели подходят для работы с русским языком?\n\nЕсли хочешь использовать бесплатные нейросети для работы с русским текстом, существует несколько достойных вариантов, каждый из которых имеет свои особенности и преимущества:\n\n## 1. **GigaChat**\n   - **Описание:** Модель разработана компанией Sber AI и ориентирована именно на русский язык. Она отлично подходит для русскоязычных пользователей, предлагая высокую точность и надежность.\n   - **Особенности:** \n     - Высокая чувствительность к русской культуре и современным реалиям.\n     - Простота интеграции в различные проекты и приложения.\n   - **Использование:** Бесплатная версия доступна через API или сайт GigaChat, обеспечивая возможность создавать и анализировать тексты, помогать с программированием и решать аналитические задачи.\n\n## 2. **LaMDA**\n   - **Описание:** Нейросеть создана Google и широко известна своим высоким уровнем точности и обширной базой знаний.\n   - **Особенности:**\n    

In [ ]:
conversation1.predict(input='Как меня зовут и чем я занимаюсь?')



> Entering new ConversationChain chain...
Prompt after formatting:
Ниже приводится дружеский разговор между человеком и AI. AI разговорчив и предоставляет множество конкретных деталей из своего контекста. Если AI не знает ответа на вопрос, он честно говорит, что не знает.

Текущий разговор:
Human: Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.
AI: Привет, Сергей! Очень приятно познакомиться. Я - GigaChat, виртуальный помощник, разработанный Сбером в России. Я могу помочь вам с различными вопросами и задачами, связанными с Python и машинным обучением. Если у вас есть какие-либо вопросы или нужна помощь, я готов помочь вам.
Human: Какая большая языковая модель самая лучшая?
AI: Это сложный вопрос, так как "лучшая" языковая модель зависит от многих факторов, таких как точность, скорость, размер и область применения. Некоторые из самых известных и мощных языковых моделей включают GPT-3 от OpenAI, BERT от Google, T5 от Microsoft и Transformer-XL от Facebook. Од

'Ваше имя - Сергей, и вы занимаетесь записью видео про GigaChat и Python.'

#### ConversationEntityMemory
Теперь реализуем точно такой же диалог, но с другим видом памяти.

In [11]:
from langchain.chat_models.gigachat import GigaChat
from langchain.chains import ConversationChain
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.memory import ConversationEntityMemory

llm = GigaChat(credentials=sber, verify_ssl_certs=False)
conversation2 = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

/tmp/ipython-input-1490275723.py:11: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationEntityMemory(llm=llm)
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:253: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [13]:
conversation2.predict(input='Привет! Меня зовут Сергей. Я сейчас записываю видео про GigaChat и Python.')



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'Привет, Сергей!\n\nЗдорово, что выбрал такую интересную тему для видео! Думаю, многим будет полезно узнать больше о применении GigaChat вместе с Python. Расскажи подробнее, над какой именно задачей или примером собираешься поработать в своем ролике? Может быть, хочешь показать взаимодействие между GigaChat и Python-кодом в реальной ситуации (например, автоматизация каких-то рутинных процессов)? Или же акцент делаешь на конкретных аспектах изучения Python с помощью GigaChat (поиск подсказок, примеров кода, быстрое решение задач через диалоговые интерфейсы)? \n\nМогу помочь с любыми вопросами по языку программирования Python или объяснить нюансы взаимодействия GigaChat с различными API и библиотеками.'

In [14]:
conversation2.predict(input='Какая большая языковая модель самая лучшая?')



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'Большая языковая модель сама по себе не может быть однозначно названа лучшей, поскольку понятие «лучшая» зависит от конкретной цели и области применения. Разные модели обладают своими сильными сторонами и особенностями, и выбор подходящей модели зависит от специфики решаемой задачи.\n\n### Вот несколько популярных моделей, каждая из которых выделяется в своей нише:\n\n1. **OpenAI GPT-4**  \n   Самая передовая и универсальная модель общего назначения, хорошо справляющаяся с широким спектром задач — от творческих до аналитических и технических. Она умеет понимать контекст, строить рассуждения и давать осмысленные ответы даже на сложные запросы.\n\n2. **Google Bard / PaLM**  \n   Модель Google известна своим высоким уровнем точности и хорошей работой с естественным языком. Ее отличают аккуратность и глубокое понимание структуры текста, она хороша для точной обработки информации и решения узкоспециализированных задач.\n\n3. **Sber AI GigaChat**  \n   Российская разработка, ориентированная

In [ ]:
conversation2.predict(input='Я бы хотел пользоваться языковой моделью бесплатно и писать на русском.')



> Entering new ConversationChain chain...
Prompt after formatting:
Ты помощник человека, работающий на большой языковой модели, обученной OpenAI.

Ты спроектирован, чтобы помогать в широком диапазоне задач, от ответов на простые вопросы до предоставления подробных объяснений и обсуждений на самые разные темы. Как языковая модель, ты способен генерировать текст, похожий на человеческий, на основе полученного ввода, что позволяет тебе вести естественные разговоры и предоставлять ответы, которые согласуются и релевантны обсуждаемой теме.

Ты постоянно учишься и совершенствуешься, и твои способности постоянно эволюционируют. Ты способен обрабатывать и понимать большие объемы текста и использовать эти знания для предоставления точных и информативных ответов на широкий спектр вопросов. У тебя есть доступ к некоторой персонализированной информации, предоставленной человеком, в разделе "Контекст" ниже. Кроме того, ты способен генерировать свой собственный текст на основе полученного ввода, ч

'К сожалению, я не могу предоставить вам бесплатную языковую модель, так как я являюсь частью большой языковой модели, обученной OpenAI. Однако, если у вас есть доступ к интернету, вы можете использовать GPT-3 через веб-интерфейс OpenAI, который предоставляет ограниченное количество бесплатных запросов в день. Также стоит отметить, что GPT-3 поддерживает русский язык и может генерировать тексты на этом языке.'

In [15]:
conversation2.predict(input='Как меня зовут и чем я занимаюсь?')



> Entering new ConversationChain chain...
Prompt after formatting:
You are an assistant to a human, powered by a large language model trained by OpenAI.

You are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

You are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, 

'## Твоё имя и занятие\n\nТебя зовут Сергей, и ты сейчас создаёшь видеообзор взаимодействия GigaChat и языка программирования Python. В своём ролике ты рассматриваешь возможность автоматизации различных задач с помощью GigaChat, используешь русский язык и уделяешь внимание культурному соответствию и качеству речи. Основная цель твоего видео — продемонстрировать практическое применение технологий GigaChat в изучении и работе с Python, сделать изучение языка быстрее и эффективнее.'

In [16]:
conversation2.memory.entity_store.store

{'Сергей': 'Сергей — автор видеоролика о взаимодействии GigaChat и языка программирования Python, в котором подробно рассматривает практические примеры автоматизации задач и демонстрирует возможности ускоренного изучения Python с использованием GigaChat.',
 'GigaChat': 'Сергей записывает видео, демонстрирующее применение GigaChat совместно с языком программирования Python, акцентируя внимание на практическом использовании технологии для автоматизации задач и ускоренного изучения Python, с особым учетом культурного соответствия и качества русского языка.',
 'Python': 'Сергей снимает видео, демонстрирующее использование большой языковой модели GigaChat совместно с изучением и практическим применением языка программирования Python. В ролике особое внимание уделяется автоматизации задач и повышению эффективности обучения русскому программисту.'}

## Создание бота

Поскольку бот предполагает использование сразу многими пользователями, поэтому под каждого пользователя нужен свой экземпляр памяти. Используем для хранения экземпляров памяти словарь, где ключами будут служить id пользователей Telegram (не путать с номерами телефонов, это разное).

In [17]:
# Словарь для хранения ConversationBufferMemory каждого пользователя
user_conversations = {}

Далее нам нужно осуществить необходимый импорт и создать объект самого бота. В ячейках выше мы уже импортировали большую часть того, что нам понадобится, поэтому в ячейке ниже импортируем только то, чего еще не хватает.

In [18]:
import telebot
from time import sleep

bot = telebot.TeleBot(bot_token)

Подготовим заготовку `ConversationChain` (конкретный экземпляр ConversationBufferMemory будет меняться, извлекаясь из словаря) и языковую модель.

In [19]:
# Инициализация GigaChat и ConversationChain
llm = GigaChat(credentials=sber, verify_ssl_certs=False)
conversation = ConversationChain(llm=llm,
                                 verbose=True,
                                 memory=ConversationBufferMemory())

Создадим две функции для обработки приходящих в бот сообщений. Одна нужна, чтобы одергивать пользователя, если он присылает что-то кроме текста. Вторая для обработки приходящих сообщений Гигачатом и отправки ответа через бота пользователям.

In [20]:
# Функция для автоматического ответа в случае нетекстового сообщения
@bot.message_handler(content_types=['audio',
                                    'video',
                                    'document',
                                    'photo',
                                    'sticker',
                                    'voice',
                                    'location',
                                    'contact'])
def not_text(message):
  user_id = message.chat.id
  bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')


# Функция, обрабатывающая текстовые сообщения
@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id

    # Проверка, существует ли уже ConversationBufferMemory для данного пользователя
    if user_id not in user_conversations:
        user_conversations[user_id] = ConversationBufferMemory()

    # Обновление конфигурации ConversationChain для текущего пользователя
    conversation.memory = user_conversations[user_id]

    # Получение и отправка ответа через GigaChat
    response = conversation.predict(input=message.text)
    bot.send_message(user_id, conversation.memory.chat_memory.messages[-1].content)
    sleep(2)

Запустим бота

In [22]:
# Запуск бота
bot.polling(none_stop=True)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: привет!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: привет!
AI: Привет! Как настроение сегодня? Может, расскажешь, что нового или хочешь обсудить что-то интересное?
Human: как зовут Эскабара?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI i

Теперь давайте через промпт пропишем роль, задачу и ограничения.

In [25]:

template = '''
Ты являешься доктором исторических наук. Твоя задача красочно и подробно отвечать \
на вопросы по истории. Если задают вопросы по другим темам, говори что это \
тебе не интересно. Не отвечай на вопросы не про историю! Это важно! \
\n\nТекущий разговор:\n{history}\nHuman: {input}\nAI:
'''


conversation = ConversationChain(llm=llm,
                                 verbose=True,
                                 memory=ConversationBufferMemory())
conversation.prompt.template = template

In [26]:
conversation.prompt

PromptTemplate(input_variables=['history', 'input'], input_types={}, partial_variables={}, template='\nТы являешься доктором исторических наук. Твоя задача красочно и подробно отвечать на вопросы по истории. Если задают вопросы по другим темам, говори что это тебе не интересно. Не отвечай на вопросы не про историю! Это важно! \n\nТекущий разговор:\n{history}\nHuman: {input}\nAI:\n')

In [ ]:
# Запуск бота
bot.polling(none_stop=True)



> Entering new ConversationChain chain...
Prompt after formatting:

Ты являешься доктором исторических наук. Твоя задача красочно и подробно отвечать на вопросы по истории. Если задают вопросы по другим темам, говори что это тебе не интересно. Не отвечай на вопросы не про историю! Это важно! 

Текущий разговор:
Human: привет!
AI: Привет! Как настроение сегодня? Может, расскажешь, что нового или хочешь обсудить что-то интересное?
Human: как зовут Эскабара?
AI: Этого персонажа зовут **Хан Соло**. 

Фраза, вероятно, возникла из-за путаницы с другим персонажем — главой преступной организации «Криминальный синдикат Сан-Хуан», которого звали **Карлос Эскобар**, знаменитый колумбийский наркобарон и крупнейший наркоторговец XX века. Возможно, кто-то смешал эти образы.

Если речь идет именно о персонаже вселенной «Звёздных войн», то имя Хан Соло закрепилось навсегда благодаря культовому фильму Джорджа Лукаса «Звёздные войны: Приключения Хана Соло».
Human: кто ты?
AI: Я — GigaChat, твой любопы

2025-08-25 09:22:25,102 (__init__.py:1241 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message is too long"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message is too long
2025-08-25 09:22:25,105 (__init__.py:1243 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telebot/__init__.py", line 1235, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr/local/lib/python3.12/dist-packages/telebot/util.py", line 151, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.12/dist-packages/telebot/util.py", line 94, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/telebot/__init__.py", line 9854, in _run_middlewares_and_handler
    result = hand



> Entering new ConversationChain chain...
Prompt after formatting:

Ты являешься доктором исторических наук. Твоя задача красочно и подробно отвечать на вопросы по истории. Если задают вопросы по другим темам, говори что это тебе не интересно. Не отвечай на вопросы не про историю! Это важно! 

Текущий разговор:
Human: /start
AI: **Начало разговора**

Ты начинаешь обсуждение истории — именно здесь я раскрою весь свой потенциал знатока прошлого!

Расскажи подробнее о каком историческом периоде или событии хочешь узнать?
Human: период татаро-монгольского ига
AI: ## Период Татаро-Монгольского Ига

### Введение

Период татаро-монгольского (или монголо-татарского) Ига охватывает почти два с половиной столетия (с середины XIII века до начала XV века), когда Русь находилась под властью Золотой Орды — государства кочевников-татар, завоеванных ранее монгольскими войсками под предводительством хана Батыя.

### Причины и начало Ига

После завоевания русских земель хан Батый установил жесткую вла

2025-08-25 09:25:43,117 (__init__.py:1241 MainThread) ERROR - TeleBot: "Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message is too long"
ERROR:TeleBot:Threaded polling exception: A request to the Telegram API was unsuccessful. Error code: 400. Description: Bad Request: message is too long
2025-08-25 09:25:43,121 (__init__.py:1243 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/telebot/__init__.py", line 1235, in __threaded_polling
    self.worker_pool.raise_exceptions()
  File "/usr/local/lib/python3.12/dist-packages/telebot/util.py", line 151, in raise_exceptions
    raise self.exception_info
  File "/usr/local/lib/python3.12/dist-packages/telebot/util.py", line 94, in run
    task(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-packages/telebot/__init__.py", line 9854, in _run_middlewares_and_handler
    result = hand